In [6]:
import os
from operator import itemgetter

from IPython.display import display, clear_output

import boto3
import hcl2


In [16]:
from helper import wait_until

# Define

In [7]:
def is_snapshot_completed(response: dict) -> bool:
    state = response['Snapshots'][0]['State']
    clear_output(wait=True)
    display(state)
    return state.lower() == 'completed'


In [8]:
TERRAFORM_FOLDER = '/terraform/eberron-agent'

In [9]:
with open(os.path.join(TERRAFORM_FOLDER, "provider.tf"), "r") as file:
    terraform = hcl2.load(file)
with open(os.path.join(TERRAFORM_FOLDER, "variables.tf"), "r") as file:
    terraform = terraform | hcl2.load(file)
for var in terraform['variable']:
    if 'region' in var.keys():
        REGION = var['region']['default']
    elif 'cluster_name' in var.keys():
        CLUSTER_NAME = var['cluster_name']['default']
    elif 'purpose' in var.keys():
        PURPOSE = var['purpose']['default']
TAGS = {'cluster': CLUSTER_NAME, 'purpose': PURPOSE}
VOLUME_FILTERS = [
    {'Name': f'tag:purpose', 
     'Values': ['kubyterlab-llm', 'llm']}]

# Instantiate

In [13]:
session = boto3.Session(region_name=REGION)
ec2_client = session.client('ec2')

# Teardown

In [10]:
%%time
!terraform -chdir=$TERRAFORM_FOLDER init -upgrade

Initializing the backend...
Upgrading modules...
- eks in .terraform/modules/eks
- eks.eks_managed_node_group in .terraform/modules/eks/modules/eks-managed-node-group
- eks.eks_managed_node_group.user_data in .terraform/modules/eks/modules/_user_data
- eks.fargate_profile in .terraform/modules/eks/modules/fargate-profile
- eks.kms in .terraform/modules/eks.kms
- eks.self_managed_node_group in .terraform/modules/eks/modules/self-managed-node-group
- eks.self_managed_node_group.user_data in .terraform/modules/eks/modules/_user_data
- vpc in .terraform/modules/vpc
Initializing provider plugins...
- Finding hashicorp/time versions matching ">= 0.9.0"...
- Finding hashicorp/null versions matching ">= 3.0.0"...
- Finding hashicorp/aws versions matching ">= 4.33.0, ~> 5.0, >= 5.46.0, >= 5.83.0"...
- Finding latest version of hashicorp/random...
- Finding hashicorp/tls versions matching ">= 3.0.0"...
- Finding hashicorp/cloudinit versions matching ">= 2.0.0"...
- Finding hashicorp/kubernetes v

In [11]:
%%time
!terraform -chdir=$TERRAFORM_FOLDER destroy -auto-approve

module.eks.module.kms.data.aws_caller_identity.current[0]: Reading...
module.vpc.aws_vpc.this[0]: Refreshing state... [id=vpc-0dce3e68b4123c27b]
aws_iam_role.eks_cluster: Refreshing state... [id=eberron-agent-eks-cluster]
module.eks.data.aws_partition.current[0]: Reading...
module.eks.data.aws_caller_identity.current[0]: Reading...
data.aws_availability_zones.available: Reading...
module.eks.module.kms.data.aws_partition.current[0]: Reading...
aws_iam_role.eks_nodes: Refreshing state... [id=EKS_Worker_Node_Role]
module.eks.aws_cloudwatch_log_group.this[0]: Refreshing state... [id=/aws/eks/eberron-agent/cluster]
module.eks.data.aws_partition.current[0]: Read complete after 0s [id=aws]
module.eks.data.aws_iam_policy_document.assume_role_policy[0]: Reading...
module.eks.module.kms.data.aws_partition.current[0]: Read complete after 0s [id=aws]
module.eks.data.aws_iam_policy_document.custom[0]: Reading...
module.eks.data.aws_iam_policy_document.assume_role_policy[0]: Read complete after 0s 

In [17]:
response = ec2_client.describe_volumes(
    Filters=VOLUME_FILTERS
)

volumes = response.get('Volumes', [])
if not volumes:
    raise RuntimeError(f'No volumes found matching the filter: {VOLUME_FILTERS}')
volume_ids = [volume['VolumeId'] for volume in volumes]
volume_ids

['vol-0270b18622c0ec3de']

In [18]:
for volume_id in volume_ids:
    response = ec2_client.create_snapshot(
        VolumeId=volume_id,
        Description=f"Snapshot For: {volume_id}. Tags: {TAGS}",
        TagSpecifications=[
            {
                'ResourceType': 'snapshot',
                'Tags': [{'Key': k, 'Value': TAGS[k]} for k in TAGS]
            }
        ]
    )
    wait_until(ec2_client.describe_snapshots, {'SnapshotIds': [response['SnapshotId']]}, is_snapshot_completed)

'completed'

In [19]:
snapshot_id = response['SnapshotId']
snapshot_id

'snap-032e242af53921532'

In [25]:
for volume_id in volume_ids:
    ec2_client.delete_volume(VolumeId=volume_id)
wait_until(ec2_client.describe_volumes, {'Filters': VOLUME_FILTERS}, lambda x: len(x['Volumes']) == 0)